In [1]:
from datetime import date, timedelta
import pandas as pd
import geopandas as gpd
import shapely.wkt

## Read earthquakes DB

In [2]:
earthquakes_df = gpd.read_file('data/earthquakes.csv').sort_values('date')
earthquakes_df

,damage,date,day,deaths,focal_depth,hour,houses_damaged,houses_destroyed,id,intensity,...,month,name,second,total_damage,total_deaths,total_houses_damaged,total_houses_destroyed,total_missing,year,geometry
0,,,,,,,,,1,,...,,"JORDAN: BAB-A-DARAA,AL-KARAK",,,,,,,-2150,None
663,,,,,,,,,717,,...,,CHINA: SHAANXI PROVINCE: HIAO YI,,,,,,,1545,None
670,,,,,,,,,7934,6.0,...,,CHINA: GUIZHOU PROVINCE: QINGLONG,,,,,,,1551,None
674,,,,,,,,,729,12.0,...,,BALKANS NW: MACEDONIA: RAZLOVCI,,,,,,,1555,None
680,,,,,,,,,7940,7.0,...,4.0,CHINA: YUNNAN PROVINCE: YILIANG,,,,,,,1560,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971,,2017-11-12,12.0,540.0,23.0,18.0,14500.0,15500.0,10275,,...,11.0,IRAN: KERMANSHAH; IRAQ: KURDISTAN,12.0,,540.0,14500.0,15500.0,,2017,None
5972,,2017-11-13,13.0,2.0,20.0,2.0,,,10276,,...,11.0,COSTA RICA: JACO,13.0,,2.0,,,,2017,None
5973,,2017-11-15,15.0,,10.0,5.0,1001.0,,10277,,...,11.0,SOUTH KOREA: POHANG,15.0,,,1001.0,,,2017,None
5974,,2017-11-18,18.0,1.0,20.0,16.0,294.0,,10282,,...,11.0,INDONESIA: NORTH MALUKU: MOROTAI,18.0,,1.0,294.0,,,2017,None


In [3]:
list(earthquakes_df)

['damage',
 'date',
 'day',
 'deaths',
 'focal_depth',
 'hour',
 'houses_damaged',
 'houses_destroyed',
 'id',
 'intensity',
 'location',
 'magnitude',
 'minute',
 'missing',
 'month',
 'name',
 'second',
 'total_damage',
 'total_deaths',
 'total_houses_damaged',
 'total_houses_destroyed',
 'total_missing',
 'year',
 'geometry']

## Format dates

In [4]:
earthquakes_df['location']

0         POINT (35.5000000000000000 31.1000000000000014)
663      POINT (111.7000000000000028 37.2000000000000028)
670      POINT (105.2000000000000028 25.8000000000000007)
674       POINT (22.8000000000000007 42.7999999999999972)
680      POINT (103.2000000000000028 24.8999999999999986)
                              ...                        
5971      POINT (45.9410000000000025 34.8860000000000028)
5972      POINT (-84.5049999999999955 9.5259999999999998)
5973     POINT (129.2700000000000102 36.0649999999999977)
5974      POINT (128.1440000000000055 2.3820000000000001)
5975    POINT (168.6829999999999927 -21.3339999999999996)
Name: location, Length: 5976, dtype: object

## Format dates

In [5]:
earthquakes_df.replace("", None, inplace=True)
earthquakes_filtered_df = earthquakes_df.dropna(subset=['year', 'month', 'day', 'location'])[['year', 'month', 'day', 'intensity', 'magnitude', 'location']]

earthquakes_filtered_df['year'] =  earthquakes_filtered_df['year'].astype(float).astype(int)
earthquakes_filtered_df['month'] = earthquakes_filtered_df['month'].astype(float).astype(int)
earthquakes_filtered_df['day'] =   earthquakes_filtered_df['day'].astype(float).astype(int)
earthquakes_filtered_df['magnitude'] =   earthquakes_filtered_df['magnitude'].astype(float)
earthquakes_filtered_df = earthquakes_filtered_df[earthquakes_filtered_df['year'] >= 1700]
earthquakes_filtered_df['date'] = earthquakes_filtered_df[["year", "month", "day"]].apply(
    lambda x: '-'.join(x.values.astype(str)), axis="columns")

earthquakes_filtered_df['date'] = pd.to_datetime(earthquakes_filtered_df['date'])

earthquakes_gdf = gpd.GeoDataFrame(earthquakes_filtered_df, crs="EPSG:4326", geometry=earthquakes_filtered_df['location'].apply(shapely.wkt.loads))


earthquakes_gdf.sort_values('date')[:10]

,year,month,day,intensity,magnitude,location,date,geometry
1057,1700,1,27,None,NaN,POINT (-125.0000000000000000 45.0000000000000000),1700-01-27,POINT (-125.00000 45.00000)
1058,1700,4,16,None,NaN,POINT (129.3000000000000114 34.2000000000000028),1700-04-16,POINT (129.30000 34.20000)
1059,1700,9,12,None,NaN,POINT (126.5000000000000000 36.5000000000000000),1700-09-12,POINT (126.50000 36.50000)
1060,1701,11,9,None,NaN,POINT (-72.4000000000000057 18.3000000000000007),1701-11-09,POINT (-72.40000 18.30000)
1061,1701,12,21,None,NaN,POINT (-97.0000000000000000 16.5000000000000000),1701-12-21,POINT (-97.00000 16.50000)
1062,1702,3,14,10.0,NaN,POINT (14.5000000000000000 41.1000000000000014),1702-03-14,POINT (14.50000 41.10000)
1064,1703,1,14,11.0,NaN,POINT (13.0999999999999996 42.7800000000000011),1703-01-14,POINT (13.10000 42.78000)
1065,1703,2,2,11.0,NaN,POINT (13.1999999999999993 42.4669999999999987),1703-02-02,POINT (13.20000 42.46700)
1066,1703,5,17,None,NaN,POINT (56.2000000000000028 26.6000000000000014),1703-05-17,POINT (56.20000 26.60000)
1067,1703,7,2,4.0,NaN,POINT (8.9169999999999998 44.3999999999999986),1703-07-02,POINT (8.91700 44.40000)


In [20]:
earthquakes_gdf.groupby(['year']).size().to_csv("earthquakes_per_year.csv")

### Earthquakes at least magnitude 7

In [12]:
major_earthquakes = earthquakes_gdf[earthquakes_gdf['magnitude'] >= 8]
major_earthquakes

,year,month,day,intensity,magnitude,location,date,geometry
1243,1755,11,1,11.0,8.5,POINT (-10.0000000000000000 37.0000000000000000),1755-11-01,POINT (-10.00000 37.00000)
2395,1896,6,15,None,8.3,POINT (144.0000000000000000 39.5000000000000000),1896-06-15,POINT (144.00000 39.50000)
2416,1897,6,12,10.0,8.0,POINT (91.0000000000000000 26.0000000000000000),1897-06-12,POINT (91.00000 26.00000)
2451,1899,9,10,11.0,8.2,POINT (-140.0000000000000000 60.0000000000000000),1899-09-10,POINT (-140.00000 60.00000)
2572,1905,7,9,None,8.4,POINT (99.0000000000000000 49.0000000000000000),1905-07-09,POINT (99.00000 49.00000)
...,...,...,...,...,...,...,...,...
5721,2012,4,11,None,8.2,POINT (92.4629999999999939 0.8020000000000000),2012-04-11,POINT (92.46300 0.80200)
5780,2013,5,24,None,8.3,POINT (153.2210000000000036 54.8920000000000030),2013-05-24,POINT (153.22100 54.89200)
5816,2014,4,1,None,8.2,POINT (-70.8169999999999931 -19.6419999999999995),2014-04-01,POINT (-70.81700 -19.64200)
5879,2015,9,16,9.0,8.3,POINT (-71.6740000000000066 -31.5730000000000004),2015-09-16,POINT (-71.67400 -31.57300)


In [19]:
major_earthquakes.groupby(['year']).size().to_csv("major_earthquakes_per_year.csv")

### Earthquakes in the PH

In [15]:
ph_loc = shapely.wkt.loads("POLYGON ((114.63134765625 2.96498436933397,129.66064453125 2.96498436933397,129.66064453125 21.2688997199677,114.63134765625 21.2688997199677,114.63134765625 2.96498436933397))")
earthquakes_in_ph = earthquakes_gdf[earthquakes_gdf['geometry'].intersects(ph_loc)]
earthquakes_in_ph[earthquakes_in_ph['year'] > 1900]

,year,month,day,intensity,magnitude,location,date,geometry
2490,1901,9,10,7.0,NaN,POINT (121.5999999999999943 14.0000000000000000),1901-09-10,POINT (121.60000 14.00000)
2493,1901,12,14,7.0,NaN,POINT (122.0000000000000000 14.0000000000000000),1901-12-14,POINT (122.00000 14.00000)
2510,1902,8,21,10.0,NaN,POINT (123.5000000000000000 7.5000000000000000),1902-08-21,POINT (123.50000 7.50000)
2512,1902,8,26,9.0,NaN,POINT (122.5999999999999943 10.8000000000000007),1902-08-26,POINT (122.60000 10.80000)
2539,1903,12,28,8.0,NaN,POINT (126.0000000000000000 7.0000000000000000),1903-12-28,POINT (126.00000 7.00000)
...,...,...,...,...,...,...,...,...
5944,2017,4,8,None,5.9,POINT (120.9290000000000020 13.7249999999999996),2017-04-08,POINT (120.92900 13.72500)
5947,2017,4,28,None,6.9,POINT (125.0930000000000035 5.5160000000000000),2017-04-28,POINT (125.09300 5.51600)
5958,2017,7,6,None,6.5,POINT (124.6329999999999956 11.1140000000000008),2017-07-06,POINT (124.63300 11.11400)
5964,2017,8,22,None,NaN,POINT (124.7360000000000042 11.0139999999999993),2017-08-22,POINT (124.73600 11.01400)


In [18]:
earthquakes_in_ph.groupby(['year']).size().to_csv("earthquake_in_ph_per_year.csv")

In [11]:
start_date = date(1900, 1, 1)
end_date = date(2017, 1, 1)
delta = timedelta(days=1)
count = 0

earthquake_flags = []

# Clear file contents
open("data/earthquakes.txt", 'w').close()

with open("data/earthquakes.txt", 'a') as f:
    while start_date <= end_date:

        if count > 0:
            f.write("\n")

        # Check if an earthquake occured on that date
        has_earthquake = earthquakes_gdf.loc[earthquakes_gdf['date'] == start_date.strftime("%Y-%m-%d")]
        f.write(str(1 if len(has_earthquake) > 0 else 0))

        # Check if a "major" earthquake occured on that date
        has_major_earthquake = great_earthquakes.loc[(major_earthquakes['date'] == start_date.strftime("%Y-%m-%d"))]
        f.write(str(1 if len(has_major_earthquake) > 0 else 0))

        # Check if an earthquake occured in the Philippines
        has_earthquake_in_location = earthquakes_in_ph.loc[(earthquakes_in_ph['date'] == start_date.strftime("%Y-%m-%d"))]
        f.write(str(1 if len(has_earthquake_in_location) > 0 else 0))
        
        count += 1
        start_date += delta